# Processing Farm 2
1000~

In [2]:
farm_start_idx = 1000
farm_end_idx = -1

In [3]:
import os
from os.path import join, exists, isfile, realpath, dirname
import argparse
from glob import glob
import random, shutil, json
from math import log10, ceil

import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler
import torchvision.models as models
from torch.utils.data.dataset import Subset

import faiss

import netvlad

import torch.optim as optim
from tensorboardX import SummaryWriter

from datetime import datetime
from os import makedirs, remove, chdir, environ

import matplotlib.pyplot as plt

import cv2

import open3d as o3d
import numpy as np
from scipy.spatial.transform import Rotation as R

import pyquaternion as pyq
import math
from tqdm import tqdm

import sys

import time

import utility

import imp

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
weight_dir = 'Jun05_17-49-03_vgg16_netvlad'
lidar_map_pcd = None

In [5]:
parser = argparse.ArgumentParser(description='pytorch-NetVlad')
parser.add_argument('--mode', type=str, default='train', help='Mode', choices=['train', 'test', 'cluster'])
parser.add_argument('--batchSize', type=int, default=4, help='Number of triplets (query, pos, negs). Each triplet consists of 12 images.')
parser.add_argument('--cacheBatchSize', type=int, default=24, help='Batch size for caching and testing')
parser.add_argument('--cacheRefreshRate', type=int, default=1000, help='How often to refresh cache, in number of queries. 0 for off')
parser.add_argument('--nEpochs', type=int, default=30, help='number of epochs to train for')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N', help='manual epoch number (useful on restarts)')
parser.add_argument('--nGPU', type=int, default=1, help='number of GPU to use.')
parser.add_argument('--optim', type=str, default='SGD', help='optimizer to use', choices=['SGD', 'ADAM'])
parser.add_argument('--lr', type=float, default=0.0001, help='Learning Rate.')
parser.add_argument('--lrStep', type=float, default=5, help='Decay LR ever N steps.')
parser.add_argument('--lrGamma', type=float, default=0.5, help='Multiply LR by Gamma for decaying.')
parser.add_argument('--weightDecay', type=float, default=0.001, help='Weight decay for SGD.')
parser.add_argument('--momentum', type=float, default=0.9, help='Momentum for SGD.')
parser.add_argument('--nocuda', action='store_true', help='Dont use cuda')
parser.add_argument('--threads', type=int, default=8, help='Number of threads for each data loader to use')
parser.add_argument('--seed', type=int, default=123, help='Random seed to use.')
parser.add_argument('--dataPath', type=str, default='/home/ubuntu/Desktop/pytorch-NetVlad/data/', help='Path for centroid data.')
parser.add_argument('--runsPath', type=str, default='/home/ubuntu/Desktop/pytorch-NetVlad/runs/', help='Path to save runs to.')
parser.add_argument('--savePath', type=str, default='checkpoints', help='Path to save checkpoints to in logdir. Default=checkpoints/')
parser.add_argument('--cachePath', type=str, default='/tmp', help='Path to save cache to.')
parser.add_argument('--resume', type=str, default='', help='Path to load checkpoint from, for resuming training or testing.')
parser.add_argument('--ckpt', type=str, default='latest', help='Resume from latest or best checkpoint.', choices=['latest', 'best'])
parser.add_argument('--evalEvery', type=int, default=1, help='Do a validation set run, and save, every N epochs.')
parser.add_argument('--patience', type=int, default=10, help='Patience for early stopping. 0 is off.')
parser.add_argument('--dataset', type=str, default='pittsburgh', help='Dataset to use', choices=['pittsburgh','naverlabs'])
parser.add_argument('--arch', type=str, default='vgg16', help='basenetwork to use', choices=['vgg16', 'alexnet'])
parser.add_argument('--vladv2', action='store_true', help='Use VLAD v2')
parser.add_argument('--pooling', type=str, default='netvlad', help='type of pooling to use', choices=['netvlad', 'max', 'avg'])
parser.add_argument('--num_clusters', type=int, default=64, help='Number of NetVlad clusters. Default=64')
parser.add_argument('--margin', type=float, default=0.1, help='Margin for triplet loss. Default=0.1')
parser.add_argument('--split', type=str, default='val', help='Data split to use for testing. Default is val', choices=['test', 'test250k', 'train', 'val'])
parser.add_argument('--fromscratch', action='store_true', help='Train from scratch rather than using pretrained models')

_StoreTrueAction(option_strings=['--fromscratch'], dest='fromscratch', nargs=0, const=True, default=False, type=None, choices=None, help='Train from scratch rather than using pretrained models', metavar=None)

In [6]:
import torch
import torchvision.transforms as transforms
import torch.utils.data as data

from os.path import join, exists
from scipy.io import loadmat
import numpy as np
from collections import namedtuple
from PIL import Image

from sklearn.neighbors import NearestNeighbors
import h5py

def input_transform():
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
    ])

def get_whole_test_set():
    return WholeDatasetFromStruct(input_transform=input_transform(), mode='test')

dbStruct = namedtuple('dbStruct', ['whichSet', 'dataset', 
                                   'db_image', 'db_utms', 'db_num', 'db_full_pose',
                                   'q_image', 'q_utms', 'q_num', 'q_full_pose',
                                   'posDistThr', 'posDistSqThr', 'nonTrivPosDistSqThr'])

class WholeDatasetFromStruct(data.Dataset):
    def __init__(self, input_transform=None, mode='train'):
        super().__init__()

        self.input_transform = input_transform

        self.dbStruct = my_parse_dbStruct(mode)
        self.images = np.hstack([self.dbStruct.db_image, self.dbStruct.q_image])

        self.whichSet = self.dbStruct.whichSet
        self.dataset = self.dbStruct.dataset

        self.positives = None
        self.distances = None

    def __getitem__(self, index):
        img = Image.open(self.images[index])
        img = img.resize((640, 480))

        if self.input_transform:
            img = self.input_transform(img)

        return img, index

    def __len__(self):
        return len(self.images)

    def getPositives(self):
        # positives for evaluation are those within trivial threshold range
        #fit NN to find them, search by radius
        if  self.positives is None:
            knn = NearestNeighbors(n_jobs=-1)
            knn.fit(self.dbStruct.db_utms)

            self.distances, self.positives = knn.radius_neighbors(self.dbStruct.q_utms, radius=self.dbStruct.posDistThr)

        return self.positives


In [7]:
def my_parse_dbStruct(_whichSet='train'):

    whichSet = _whichSet
    dataset = 'naverlabs'
    
     # for (2019-04-16_14-35-00)
    image_path = '/home/ubuntu/Desktop/visual-localization-challenge-2020/indoor_dataset/1f/train/2019-04-16_14-35-00/images'
    image_files_list = []
    image_files_list.append(sorted(glob(os.path.join(image_path, '22970285*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '22970286*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '22970288*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '22970289*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '22970290*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '22970291*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324954*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324955*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324968*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324969*.jpg'))))
    
    gt_path = '/home/ubuntu/Desktop/visual-localization-challenge-2020/indoor_dataset/1f/train/2019-04-16_14-35-00/groundtruth.hdf5'
    full_pose_list = []
    with h5py.File(gt_path, "r") as f:
        full_pose_list.append(np.array(f['22970285_pose']))
        full_pose_list.append(np.array(f['22970286_pose']))
        full_pose_list.append(np.array(f['22970288_pose']))
        full_pose_list.append(np.array(f['22970289_pose']))
        full_pose_list.append(np.array(f['22970290_pose']))
        full_pose_list.append(np.array(f['22970291_pose']))
        full_pose_list.append(np.array(f['AC01324954_pose']))
        full_pose_list.append(np.array(f['AC01324955_pose']))
        full_pose_list.append(np.array(f['AC01324968_pose']))
        full_pose_list.append(np.array(f['AC01324969_pose']))
    
    
    # for (2019-08-20_11-32-05)
    image_path = '/home/ubuntu/Desktop/visual-localization-challenge-2020/indoor_dataset/1f/train/2019-08-20_11-32-05/images'
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324954*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324955*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324968*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324969*.jpg'))))
    
    gt_path = '/home/ubuntu/Desktop/visual-localization-challenge-2020/indoor_dataset/1f/train/2019-08-20_11-32-05/groundtruth.hdf5'
    with h5py.File(gt_path, "r") as f:
        full_pose_list.append(np.array(f['AC01324954_pose']))
        full_pose_list.append(np.array(f['AC01324955_pose']))
        full_pose_list.append(np.array(f['AC01324968_pose']))
        full_pose_list.append(np.array(f['AC01324969_pose']))
    
    db_image = np.hstack(image_files_list)
    db_full_pose = np.vstack(full_pose_list)
    db_num = len(db_image)
    db_utms = db_full_pose[:,:2]
    
    image_path = '/home/ubuntu/Desktop/visual-localization-challenge-2020/indoor_dataset/1f/test/2019-08-21_12-10-13/images'
    image_files_list = []
    image_files_list.append(sorted(glob(os.path.join(image_path, '40027089*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '40029628*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '40030065*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '40031951*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '40033113*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, '40033116*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324954*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324955*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324968*.jpg'))))
    image_files_list.append(sorted(glob(os.path.join(image_path, 'AC01324969*.jpg'))))
    image_files = np.hstack(image_files_list)
    
    ####### for processing farm
    image_files = image_files[farm_start_idx:farm_end_idx]
    
    q_image = image_files
    q_utms = None
    q_num = len(q_image)
    q_full_pose = None
    

    return dbStruct(whichSet, dataset, 
                    db_image, db_utms, db_num, db_full_pose,
                    q_image, q_utms, q_num, q_full_pose, 
                    5, 25, 20.0)

In [8]:
opt = parser.parse_args(args=('--dataset=naverlabs --mode=test --resume=runs/%s --split=test' % weight_dir).split(' '))
restore_var = ['lr', 'lrStep', 'lrGamma', 'weightDecay', 'momentum', 
            'runsPath', 'savePath', 'arch', 'num_clusters', 'pooling', 'optim',
            'margin', 'seed', 'patience']

flag_file = join(opt.resume, 'checkpoints', 'flags.json')
if exists(flag_file):
    with open(flag_file, 'r') as f:
        stored_flags = {'--'+k : str(v) for k,v in json.load(f).items() if k in restore_var}
        to_del = []
        for flag, val in stored_flags.items():
            for act in parser._actions:
                if act.dest == flag[2:]:
                    # store_true / store_false args don't accept arguments, filter these 
                    if type(act.const) == type(True):
                        if val == str(act.default):
                            to_del.append(flag)
                        else:
                            stored_flags[flag] = ''
        for flag in to_del: del stored_flags[flag]

        train_flags = [x for x in list(sum(stored_flags.items(), tuple())) if len(x) > 0]
        print('Restored flags:', train_flags)
        opt = parser.parse_args(train_flags, namespace=opt)




Restored flags: ['--optim', 'SGD', '--lr', '0.0001', '--lrStep', '5', '--lrGamma', '0.5', '--weightDecay', '0.001', '--momentum', '0.9', '--seed', '123', '--runsPath', '/home/ubuntu/Desktop/pytorch-NetVlad/runs/', '--savePath', '/home/ubuntu/Desktop/pytorch-NetVlad/runs/Jun05_17-49-03_vgg16_netvlad/checkpoints', '--patience', '10', '--arch', 'vgg16', '--pooling', 'netvlad', '--num_clusters', '64', '--margin', '0.1']


In [9]:
print(opt)

Namespace(arch='vgg16', batchSize=4, cacheBatchSize=24, cachePath='/tmp', cacheRefreshRate=1000, ckpt='latest', dataPath='/home/ubuntu/Desktop/pytorch-NetVlad/data/', dataset='naverlabs', evalEvery=1, fromscratch=False, lr=0.0001, lrGamma=0.5, lrStep=5.0, margin=0.1, mode='test', momentum=0.9, nEpochs=30, nGPU=1, nocuda=False, num_clusters=64, optim='SGD', patience=10, pooling='netvlad', resume='runs/Jun05_17-49-03_vgg16_netvlad', runsPath='/home/ubuntu/Desktop/pytorch-NetVlad/runs/', savePath='/home/ubuntu/Desktop/pytorch-NetVlad/runs/Jun05_17-49-03_vgg16_netvlad/checkpoints', seed=123, split='test', start_epoch=0, threads=8, vladv2=False, weightDecay=0.001)


In [10]:
cuda = True
device = torch.device("cuda")
random.seed(opt.seed)
np.random.seed(opt.seed)
torch.manual_seed(opt.seed)
torch.cuda.manual_seed(opt.seed)

In [11]:
whole_test_set = get_whole_test_set()

In [12]:
encoder_dim = 512
encoder = models.vgg16(pretrained=True)
# capture only feature part and remove last relu and maxpool
layers = list(encoder.features.children())[:-2]

# if using pretrained then only train conv5_1, conv5_2, and conv5_3
for l in layers[:-5]: 
    for p in l.parameters():
        p.requires_grad = False

In [13]:
encoder = nn.Sequential(*layers)
model = nn.Module() 
model.add_module('encoder', encoder)

In [14]:
net_vlad = netvlad.NetVLAD(num_clusters=opt.num_clusters, dim=encoder_dim, vladv2=opt.vladv2)
if not opt.resume: 
    if opt.mode.lower() == 'train':
        initcache = join(opt.dataPath, 'centroids', opt.arch + '_' + train_set.dataset + '_' + str(opt.num_clusters) +'_desc_cen.hdf5')
    else:
        initcache = join(opt.dataPath, 'centroids', opt.arch + '_' + whole_test_set.dataset + '_' + str(opt.num_clusters) +'_desc_cen.hdf5')

    if not exists(initcache):
        raise FileNotFoundError('Could not find clusters, please run with --mode=cluster before proceeding')

    with h5py.File(initcache, mode='r') as h5: 
        clsts = h5.get("centroids")[...]
        traindescs = h5.get("descriptors")[...]
        net_vlad.init_params(clsts, traindescs) 
        del clsts, traindescs

model.add_module('pool', net_vlad)

In [15]:
if opt.ckpt.lower() == 'latest':
    resume_ckpt = join(opt.resume, 'checkpoints', 'checkpoint.pth.tar')
elif opt.ckpt.lower() == 'best':
    resume_ckpt = join(opt.resume, 'checkpoints', 'model_best.pth.tar')

if isfile(resume_ckpt):
    print("=> loading checkpoint '{}'".format(resume_ckpt))
    checkpoint = torch.load(resume_ckpt, map_location=lambda storage, loc: storage)
    opt.start_epoch = checkpoint['epoch']
    best_metric = checkpoint['best_score']
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)
    if opt.mode == 'train':
        optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})"
          .format(resume_ckpt, checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(resume_ckpt))

=> loading checkpoint 'runs/Jun05_17-49-03_vgg16_netvlad/checkpoints/checkpoint.pth.tar'
=> loaded checkpoint 'runs/Jun05_17-49-03_vgg16_netvlad/checkpoints/checkpoint.pth.tar' (epoch 30)


# Calculate the prediction

In [16]:
test_set = whole_test_set
test_data_loader = DataLoader(dataset=test_set, num_workers=opt.threads, batch_size=opt.cacheBatchSize, shuffle=True, pin_memory=cuda)

In [17]:
start = time.time() 

model.eval()
with torch.no_grad():
    print('====> Extracting Features')
    pool_size = encoder_dim
    if opt.pooling.lower() == 'netvlad': pool_size *= opt.num_clusters
    dbFeat = np.empty((len(test_set), pool_size))

    for iteration, (input, indices) in enumerate(test_data_loader, 1):
        input = input.to(device)
        image_encoding = model.encoder(input)
        vlad_encoding = model.pool(image_encoding) 

        dbFeat[indices.detach().numpy(), :] = vlad_encoding.detach().cpu().numpy()
        if iteration % 50 == 0 or len(test_data_loader) <= 10:
            print("==> Batch ({}/{})".format(iteration, len(test_data_loader)), flush=True)

        del input, image_encoding, vlad_encoding
del test_data_loader

print("end time :", time.time() - start)

====> Extracting Features
==> Batch (50/1176)
==> Batch (100/1176)
==> Batch (150/1176)
==> Batch (200/1176)
==> Batch (250/1176)
==> Batch (300/1176)
==> Batch (350/1176)
==> Batch (400/1176)
==> Batch (450/1176)
==> Batch (500/1176)
==> Batch (550/1176)
==> Batch (600/1176)
==> Batch (650/1176)
==> Batch (700/1176)
==> Batch (750/1176)
==> Batch (800/1176)
==> Batch (850/1176)
==> Batch (900/1176)
==> Batch (950/1176)
==> Batch (1000/1176)
==> Batch (1050/1176)
==> Batch (1100/1176)
==> Batch (1150/1176)
end time : 395.7727994918823


In [18]:
start = time.time() 

# extracted for both db and query, now split in own sets
qFeat = dbFeat[test_set.dbStruct.db_num:].astype('float32')
dbFeat = dbFeat[:test_set.dbStruct.db_num].astype('float32')

print('====> Building faiss index')
faiss_index = faiss.IndexFlatL2(pool_size)
faiss_index.add(dbFeat)

print('====> Calculating recall @ N')
n_values = [1,5,10,20]

_, predictions = faiss_index.search(qFeat, max(n_values)) 

print("end time :", time.time() - start)

====> Building faiss index
====> Calculating recall @ N
end time : 40.15509104728699


# SuperPoint & SuperGlue

In [19]:
from super_models.matching import Matching
from super_models.utils import (AverageTimer, VideoStreamer,make_matching_plot_fast, frame2tensor)
import matplotlib.cm as cm

In [20]:
parser = argparse.ArgumentParser(description='SuperGlue demo',formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument(
        '--input', type=str, default='0',
        help='ID of a USB webcam, URL of an IP camera, '
             'or path to an image directory or movie file')
parser.add_argument(
    '--output_dir', type=str, default=None,
    help='Directory where to write output frames (If None, no output)')

parser.add_argument(
    '--image_glob', type=str, nargs='+', default=['*.png', '*.jpg', '*.jpeg'],
    help='Glob if a directory of images is specified')
parser.add_argument(
    '--skip', type=int, default=1,
    help='Images to skip if input is a movie or directory')
parser.add_argument(
    '--max_length', type=int, default=1000000,
    help='Maximum length if input is a movie or directory')
parser.add_argument(
    '--resize', type=int, nargs='+', default=[640, 480],
    help='Resize the input image before running inference. If two numbers, '
         'resize to the exact dimensions, if one number, resize the max '
         'dimension, if -1, do not resize')

parser.add_argument(
    '--superglue', choices={'indoor', 'outdoor'}, default='indoor',
    help='SuperGlue weights')
parser.add_argument(
    '--max_keypoints', type=int, default=-1,
    help='Maximum number of keypoints detected by Superpoint'
         ' (\'-1\' keeps all keypoints)')
parser.add_argument(
    '--keypoint_threshold', type=float, default=0.005,
    help='SuperPoint keypoint detector confidence threshold')
parser.add_argument(
    '--nms_radius', type=int, default=4,
    help='SuperPoint Non Maximum Suppression (NMS) radius'
    ' (Must be positive)')
parser.add_argument(
    '--sinkhorn_iterations', type=int, default=20,
    help='Number of Sinkhorn iterations performed by SuperGlue')
parser.add_argument(
    '--match_threshold', type=float, default=0.2,
    help='SuperGlue match threshold')

parser.add_argument(
    '--show_keypoints', action='store_true',
    help='Show the detected keypoints')
parser.add_argument(
    '--no_display', action='store_true',
    help='Do not display images to screen. Useful if running remotely')
parser.add_argument(
    '--force_cpu', action='store_true',
    help='Force pytorch to run in CPU mode.')

opt = parser.parse_args('--resize -1 --max_keypoints 3000'.split(' '))
# opt = parser.parse_args('')
print(opt)

Namespace(force_cpu=False, image_glob=['*.png', '*.jpg', '*.jpeg'], input='0', keypoint_threshold=0.005, match_threshold=0.2, max_keypoints=3000, max_length=1000000, nms_radius=4, no_display=False, output_dir=None, resize=[-1], show_keypoints=False, sinkhorn_iterations=20, skip=1, superglue='indoor')


In [21]:
device = 'cuda'
print('Running inference on device \"{}\"'.format(device))

config = {
    'superpoint': {
        'nms_radius': opt.nms_radius,
        'keypoint_threshold': opt.keypoint_threshold,
        'max_keypoints': opt.max_keypoints
    },
    'superglue': {
        'weights': opt.superglue,
        'sinkhorn_iterations': opt.sinkhorn_iterations,
        'match_threshold': opt.match_threshold,
    }
}
print(config)

Running inference on device "cuda"
{'superpoint': {'nms_radius': 4, 'keypoint_threshold': 0.005, 'max_keypoints': 3000}, 'superglue': {'weights': 'indoor', 'sinkhorn_iterations': 20, 'match_threshold': 0.2}}


In [22]:
def process_resize(w, h, resize):
    assert(len(resize) > 0 and len(resize) <= 2)
    if len(resize) == 1 and resize[0] > -1:
        scale = resize[0] / max(h, w)
        w_new, h_new = int(round(w*scale)), int(round(h*scale))
    elif len(resize) == 1 and resize[0] == -1:
        w_new, h_new = w, h
    else:  # len(resize) == 2:
        w_new, h_new = resize[0], resize[1]

    # Issue warning if resolution is too small or too large.
    if max(w_new, h_new) < 160:
        print('Warning: input resolution is very small, results may vary')
    elif max(w_new, h_new) > 2000:
        print('Warning: input resolution is very large, results may vary')

    return w_new, h_new

def load_image(impath, resize):
    """ Read image as grayscale and resize to img_size.
    Inputs
        impath: Path to input image.
    Returns
        grayim: uint8 numpy array sized H x W.
    """

    camera_model = impath.split('/')[-1].split('_')[0]
    _, A, dist_coeff = utility.intrinsic_params(camera_model)
    
    grayim = cv2.imread(impath, 0)
    grayim = cv2.undistort(grayim, A, dist_coeff)
    if grayim is None:
        raise Exception('Error reading image %s' % impath)
    w, h = grayim.shape[1], grayim.shape[0]
    w_new, h_new = process_resize(w, h, resize)
    grayim = cv2.resize(grayim, (w_new, h_new), interpolation=cv2.INTER_AREA)
    return grayim

In [23]:
def super_matching(matching, query_path, pred_path):
    
    keys = ['keypoints', 'scores', 'descriptors']

    frame = load_image(query_path, opt.resize)
    frame_tensor = frame2tensor(frame, device)
    last_data = matching.superpoint({'image': frame_tensor})
    last_data = {k+'0': last_data[k] for k in keys}
    last_data['image0'] = frame_tensor
    last_frame = frame
    last_image_id = 0

    frame = load_image(pred_path, opt.resize)
    frame_tensor = frame2tensor(frame, device)
    pred = matching({**last_data, 'image1': frame_tensor})
    kpts0 = last_data['keypoints0'][0].cpu().numpy()
    kpts1 = pred['keypoints1'][0].cpu().numpy()
    matches = pred['matches0'][0].cpu().numpy()
    confidence = pred['matching_scores0'][0].cpu().detach().numpy()

    valid = matches > -1
    mkpts0 = kpts0[valid]
    mkpts1 = kpts1[matches[valid]]
    color = cm.jet(confidence[valid])
    text = [
        'SuperGlue',
        'Keypoints: {}:{}'.format(len(kpts0), len(kpts1)),
        'Matches: {}'.format(len(mkpts0))
    ]
    k_thresh = matching.superpoint.config['keypoint_threshold']
    m_thresh = matching.superglue.config['match_threshold']
    
    small_text = [
        'Keypoint Threshold: {:.4f}'.format(k_thresh),
        'Match Threshold: {:.2f}'.format(m_thresh),
    ]
    matching_img = make_matching_plot_fast(
        last_frame, frame, kpts0, kpts1, mkpts0, mkpts1, color, text,
        path=None, show_keypoints=False, small_text=small_text)

    return matching_img, mkpts0, mkpts1

# Load HD Map

In [24]:
start = time.time() 

# VERY TIME CONSUMING!
lidar_map_pcd1 = o3d.io.read_point_cloud("/home/ubuntu/Desktop/visual-localization-challenge-2020/indoor_dataset/1f/train/2019-04-16_14-35-00/map.pcd")
pcd_tree1 = o3d.geometry.KDTreeFlann(lidar_map_pcd1)

lidar_map_pcd2 = o3d.io.read_point_cloud("/home/ubuntu/Desktop/visual-localization-challenge-2020/indoor_dataset/1f/train/2019-08-20_11-32-05/map.pcd")
pcd_tree2 = o3d.geometry.KDTreeFlann(lidar_map_pcd2)

print("end time :", time.time() - start)

end time : 346.5583760738373


# All-in-one

In [25]:
imp.reload(utility)

def VisualLocalization(dataset, query_idx):
    query_item, pred_list = utility.placeRecognitionTopFive(dataset, predictions, query_idx)
    [query_image_full_path, query_img, _] = query_item
    query_camera_model = query_image_full_path.split('/')[-1].split('_')[0]
    _, query_A, query_dist_coeff = utility.intrinsic_params(query_camera_model)

    neighbor_radius = 20.0    

    best_inlier = 0
    best_pose = []
    for rank, pred_item in enumerate(pred_list):
        [pred_image_full_path, pred_img, pred_pose] = pred_item

        acquisition_date = pred_image_full_path.split('/')[-3] 
        if acquisition_date == '2019-04-16_14-35-00':
            pcd = lidar_map_pcd1
            pcd_tree = pcd_tree1
        elif acquisition_date == '2019-08-20_11-32-05':
            pcd = lidar_map_pcd2
            pcd_tree = pcd_tree2
            
        [k, idx, _] = pcd_tree.search_radius_vector_3d(pred_pose[:3], neighbor_radius) # time consuming
        radius_points = np.asarray(pcd.points)[idx] 

        pred_Rt = np.eye(4)
        pred_Rt[:3,3] = pred_pose[:3]
        (pred_qw, pred_qx, pred_qy, pred_qz) = pred_pose[3:]
        pred_Rt[:3,:3] = R.from_quat([pred_qx,pred_qy,pred_qz,pred_qw]).as_matrix()
        pred_camera_model = pred_image_full_path.split('/')[-1].split('_')[0]
        _, pred_A, pred_dist_coeff = utility.intrinsic_params(pred_camera_model)
        pred_projected_img = utility.projection(pred_img, radius_points, pred_A, pred_Rt, thickness=10) # time consuming 
    
        # super matching
        matching = Matching(config).eval().to(device)
        matching_img, px_query, px_pred = super_matching(matching, query_item[0], pred_item[0])
        match_size = len(px_query)
        print('match_size : ', match_size)
        del matching
        torch.cuda.empty_cache()

        # pnp
        points_2d = []
        points_3d = []
        for corr_idx in range(match_size):
            if (not np.array_equal(pred_projected_img[int(px_pred[corr_idx][1]), int(px_pred[corr_idx][0])], np.ones(3)*np.inf)):
                points_2d.append([px_query[corr_idx][0],px_query[corr_idx][1]])
                points_3d.append(pred_projected_img[int(px_pred[corr_idx][1]), int(px_pred[corr_idx][0])])

        points_2d = np.asarray(points_2d, dtype=np.float32)
        points_3d = np.asarray(points_3d, dtype=np.float32)

        try:
            retval, rvec, tvec, inliers = cv2.solvePnPRansac(points_3d, points_2d, query_A, query_dist_coeff)
        except:
            continue

        if (retval):
            rotation_matrix = np.zeros(shape=(3,3))
            cv2.Rodrigues(rvec, rotation_matrix)
            query_rot = pred_Rt[:3,:3]@np.linalg.inv(np.asarray(rotation_matrix))
            query_trans = np.transpose(np.asarray(tvec)) + pred_Rt[:3,3]
            query_Rt = np.eye(4)
            query_Rt[:3,:3] = query_rot
            query_Rt[:3,3] = query_trans

            query_quat = R.from_matrix(query_rot).as_quat()
            result_pose = (query_quat,query_trans)

            if (best_inlier < len(inliers)):
                best_inlier = len(inliers)
                best_pose = result_pose
            
    
    if (best_inlier == 0):
        return False, None, None, None
    else:
        return True, query_image_full_path, query_img, best_pose

# Test One

In [26]:
start = time.time()
query_idx = np.random.randint(test_set.dbStruct.q_num)
ret_val, query_image_full_path, query_img, best_pose = VisualLocalization(test_set, query_idx)
print("time :", time.time() - start)

NameError: name 'query_idx' is not defined

In [ ]:
neighbor_radius = 20
pcd = lidar_map_pcd1
pcd_tree = pcd_tree1

[k, idx, _] = pcd_tree.search_radius_vector_3d(best_pose[1][0], neighbor_radius) # time consuming
radius_points = np.asarray(pcd.points)[idx] 

query_Rt = np.eye(4)
query_Rt[:3,3] = best_pose[1][0]
(query_qx, query_qy, query_qz, query_qw) = best_pose[0]
query_Rt[:3,:3] = R.from_quat([query_qx, query_qy, query_qz,query_qw]).as_matrix()
query_camera_model = query_image_full_path.split('/')[-1].split('_')[0]
_, query_A, query_dist_coeff = utility.intrinsic_params(query_camera_model)

query_projected_img = utility.projection(query_img, radius_points, query_A, query_Rt, thickness=3) # time consuming 

In [ ]:
plt.figure()
plt.imshow(query_projected_img[:,:,2])
plt.figure()
plt.imshow(query_img)

In [ ]:
temp_result_list = []
temp_dict = {
         'floor': "1f",
         'name': query_image_full_path.split('/')[-1],
         'qw': best_pose[0][3],
         'qx': best_pose[0][0],
         'qy': best_pose[0][1],
         'qz': best_pose[0][2],
         'x': best_pose[1][0][0],
         'y': best_pose[1][0][1],
         'z': best_pose[1][0][2]
}

temp_result_list.append(temp_dict)

In [ ]:
with open('20200701_indoor_submit (temp).json', 'w') as outfile:
    json.dump(temp_result_list, outfile)

# Test ALL

In [ ]:
result_list = []
for query_idx in tqdm(range(test_set.dbStruct.q_num)):
    
    try:
        ret_val, query_image_full_path , _, best_pose = VisualLocalization(test_set, query_idx)
    except KeyboardInterrupt: 
        break
    except: 
        continue
        
    if ret_val == True:        
        temp_dict = {
                 'floor': "1f",
                 'name': query_image_full_path.split('/')[-1],
                 'qw': best_pose[0][3],
                 'qx': best_pose[0][0],
                 'qy': best_pose[0][1],
                 'qz': best_pose[0][2],
                 'x': best_pose[1][0][0],
                 'y': best_pose[1][0][1],
                 'z': best_pose[1][0][2]
        }

        result_list.append(temp_dict)

    if (query_idx % 10 == 0):
        with open('20200701_indoor_submit (farm2).json', 'w') as outfile:
            json.dump(result_list, outfile)

  0%|          | 0/968 [00:00<?, ?it/s]

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  667
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  340
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  0
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  7
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


  0%|          | 1/968 [01:00<16:20:51, 60.86s/it]

match_size :  66
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  432
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  621
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  1180
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  460
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


  0%|          | 2/968 [02:08<16:52:37, 62.90s/it]

...

match_size :  596
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  653
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  518
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  434
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  419
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


 46%|████▌     | 446/968 [8:09:13<9:39:18, 66.59s/it] 

match_size :  59
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  1119
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  705
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  150
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  25
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


 46%|████▌     | 447/968 [8:09:51<8:23:12, 57.95s/it]

match_size :  359
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  887
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  886
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
match_size :  901
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


In [29]:
with open('20200701_indoor_submit (farm2).json', 'w') as outfile:
    json.dump(result_list, outfile)